In [1]:
import ee
import os
import earthpy as et


import geemap as emap
import folium
import geopandas as gpd
import time


try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [2]:
# Set working directory

# if the desired path exists:
data_dir = os.path.join(et.io.HOME, 'Dropbox',
                        'cu_earthdata_certificate_2021', 'earthlab_project', 'data')
if os.path.exists(data_dir):
    # set working directory:
    os.chdir(data_dir)
    print("path exists")
else:
    print("path does not exist, making new path")
    os.makedirs(data_dir)
    os.chdir(data_dir)

path exists


In [3]:
# Import file containing full catchment for each burned lake
# from local drive generated in 01_download_clip_merge.ipynb
file_path = os.path.join('burned_w_aquasat_polys.geojson')

# Geodataframe used in folium map
lake_catchmts = gpd.read_file(file_path).to_crs(epsg=4326)

# EE object used for Earth Engine collections and calculations
small_gdf = lake_catchmts[["Hylak_id", "geometry"]]
lake_ctchmts_ee = emap.geopandas_to_ee(small_gdf)

In [4]:
# Define a function to calculate NBR.
def add_nbr(img):
    nbr = img.normalizedDifference(['B4', 'B7']).multiply(1000).select([0], ['NBR']);
    return img.addBands(nbr)

# Define a function to calculate NDVI.
def add_ndvi(img):
    ndvi = img.normalizedDifference(['B5', 'B4']).multiply(1000).select([0], ['NDVI']);
    return img.addBands(ndvi);
    #return image.normalizedDifference(['B5', 'B4']).rename('NDVI')



In [14]:
# This function adds a band representing the image timestamp.
def addTime(image): 
    return image.addBands(image.metadata('system:time_start'))

# Define a function to add year as an image property.
def set_year(img):
    year = ee.Image(img).date().get('year')
    return img.addBands(year);

In [15]:
# Define a function to mask out clouds and cloud shadows.
def cfmask(img):
    cloud_shadow_bi_mask = 1 << 3
    cloud_bit_mask = 1 << 5
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloud_shadow_bi_mask).eq(0).And(
      qa.bitwiseAnd(cloud_bit_mask).eq(0))
    return img.updateMask(mask)



In [16]:
def l8cloudmask(img):
    fillBitMask = 1
    clearBitMask = 1 << 1
    waterBitMask = 1 << 2
    cloudShadowBitMask = 1 << 3
    snowBitMask = 1 << 4
    cloudConfBitMask = 1 << 7
    cirrusConfBitMask = 1 << 9
    occlusionBitMask = 1 << 10

    ls_bands = ee.List(['B4', 'B5', 'B6'])
    pixelqa_name = 'BQA'

  # Get the pixel QA band
    pixelqa = img.select(pixelqa_name)

    mask = pixelqa.bitwiseAnd(fillBitMask).eq(0).And(pixelqa.bitwiseAnd(clearBitMask).neq(0)).And(pixelqa.bitwiseAnd(waterBitMask).eq(0)).And(pixelqa.bitwiseAnd(cloudShadowBitMask).eq(0)).And(pixelqa.bitwiseAnd(snowBitMask).eq(0)).And(pixelqa.bitwiseAnd(cloudConfBitMask).eq(0)).And(pixelqa.bitwiseAnd(cirrusConfBitMask).eq(0)).And(pixelqa.bitwiseAnd(occlusionBitMask).eq(0))  
# Return the masked image (excluding the PIXELQA and STQA layers)
    return img.updateMask(mask).select(ls_bands)

In [17]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
    """Extracts the values from a FeatureCollection and stores them as a list in ee.Dictionary. ee.Dictionary
    can be converted to a pandasDataframe if needed. 
    
    Args:
    ee.featureCollection:
      An ee.featureCollection contains images and properties that can be converted 
      to a dictionary with geometries. 
      
    Returns:
    A ee.Dictionary (useful for converting to Pandas objects)
    """
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

    return ee.Dictionary.fromLists(prop_names, prop_lists)

In [18]:
point = ee.Geometry.Point(-83.93, 35.85)

In [19]:
# select three Landsat 8 imagery in the Southwest area with the least cloud coverage
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterDate('2017-03-01','2017-04-30')\
    .select(['B4','B5', 'B6','B7','pixel_qa'])\
    .filterBounds(lake_ctchmts_ee)

# calculate NDVI for each image using the map fucntion    
vi_images = collection.map(addTime)
vi_images = vi_images.map(cfmask)

# calculate VI for each image using the map fucntion    
vi_images = vi_images.map(add_ndvi)
vi_images = vi_images.map(add_nbr)
vi_images = vi_images.map(set_year)

vi_median = vi_images.median()


ls8_reduced_mean = ee.FeatureCollection(vi_median.reduceRegions(reducer=ee.Reducer.mean(),
                                        collection=lake_ctchmts_ee,
                                        scale=30))



ls8_reduced_mean.aggregate_stats('mean').getInfo()

{'max': None,
 'mean': 0,
 'min': None,
 'sample_sd': 0,
 'sample_var': 0,
 'sum': 0,
 'sum_sq': 0,
 'total_count': 0,
 'total_sd': 0,
 'total_var': 0,
 'valid_count': 0,
 'weight_sum': 0,
 'weighted_sum': 0}

In [20]:
task = ee.batch.Export.table.toDrive(**{
    'collection': ls8_reduced_mean,
    'description': 'vi_mock_export',
    'folder':'new_example_folder',
    'fileFormat': 'CSV'
})

task.start()

In [21]:
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(15)

Polling for task (id: 6MPUO3LYRK7TZPBNNZQ3TV3U).
Polling for task (id: 6MPUO3LYRK7TZPBNNZQ3TV3U).


In [24]:
Map = emap.Map(center=[37,-105], zoom=8)
Map.add_basemap('ROADMAP') # Add Google Map
Map


vizParams = {
    'bands': ['NDVI', 'NBR', 'B6'],
    'min': 0, 
    'max': 0.4
}

#Map.centerObject(lake_ctchmts_ee, 8)

Map.addLayer(vi_median, vizParams, 'Landsat image ')

Map.addLayer(lake_ctchmts_ee, {}, 'burnedlakes')
    
Map.addLayerControl()
Map

Map(center=[37, -105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…